In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '/cellar/users/mikeyu/DeepTranslate/ddot')

import numpy as np
np.set_printoptions(linewidth=150)

# Introduction: DDOT tutorial

* __What is an ontology?__ An ontology is a hierarchical arrangement of two types of nodes: (1)
genes at the leaves of the hierarchy and (2) terms at intermediate
levels of the hierarchy. The hierarchy can be thought of as directed
acyclic graph (DAG), in which each node can have multiple children or
multiple parent nodes. DAGs are a generalization of trees
(a.k.a. dendogram), where each node has at most one parent.

* __What is DDOT?__ The DDOT Python package provides many functions for assembling,
analyzing, and visualizing ontologies.  The main functionalities are
implemented in an object-oriented manner by an "Ontology" class, which handles ontologies that are data-driven as well as those
that are manually curated like the Gene Ontology.

* __What to do after reading this tutorial__ Check out a complete list of functions in the [Ontology class](http://ddot.readthedocs.io/en/latest/ontology.html) and a list of [utility functions](http://ddot.readthedocs.io/en/latest/utils.html) that may help you build more concise pipelines. Also check out [example Jupyter notebooks](https://github.com/michaelkyu/ddot/tree/master/examples) that contain pipelines for downloading and processing the Gene Ontology and for inferring data-driven gene ontologies of diseases

In [1]:
# Import Ontology class from DDOT package
import ddot
from ddot import Ontology

/cellar/users/mikeyu/anaconda2/envs/ddot_py36/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.23) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


# Creating an Ontology object

* An object of the Ontology class can be created in several ways.
* In this tutorial, we will construct and analyze the toy ontology shown below.

<img src="https://raw.githubusercontent.com/michaelkyu/ddot/master/docs/toy_ontology.png" height="250" align="left">

### Create ontology through the \_\_init\_\_ constructor

In [2]:
# Connections from child terms to parent terms
hierarchy = [('S3', 'S1'),
             ('S4', 'S1'),
             ('S5', 'S1'),
             ('S5', 'S2'),
             ('S6', 'S2'),
             ('S1', 'S0'),
             ('S2', 'S0')]

# Connections from genes to terms
mapping = [('A', 'S3'),
           ('B', 'S3'),
           ('C', 'S3'),
           ('C', 'S4'),
           ('D', 'S4'),
           ('E', 'S5'),
           ('F', 'S5'),
           ('G', 'S6'),
           ('H', 'S6')]

# Construct ontology
ont = Ontology(hierarchy, mapping)

# Prints a summary of the ontology's structure
print(ont)

8 genes, 7 terms, 9 gene-term relations, 7 term-term relations
node_attributes: []
edge_attributes: []


### Create an ontology from a tab-delimited table or Pandas dataframe

In [3]:
# Write ontology to a tab-delimited table
ont.to_table('toy_ontology.txt')

,Parent,Child,EdgeType
0,S0,S1,Child-Parent
1,S0,S2,Child-Parent
2,S1,S3,Child-Parent
3,S1,S4,Child-Parent
4,S1,S5,Child-Parent
5,S2,S5,Child-Parent
6,S2,S6,Child-Parent
7,S3,A,Gene-Term
8,S3,B,Gene-Term
9,S4,C,Gene-Term


In [4]:
# Reconstruct the ontology from the table
ont2 = Ontology.from_table('toy_ontology.txt')
ont2

8 genes, 7 terms, 9 gene-term relations, 7 term-term relations
node_attributes: []
edge_attributes: [2]

### From the Network Data Exchange (NDEx).
* It is strongly recommended that you create a free account on NDEx in order to keep track of your own ontologies.
* Note that there are two NDEx servers: the main one at http://ndexbio.org/ and a test server for prototyping your code at http://test.ndexbio.org. Each server requires a separate user account. While you get familiar with DDOT, we recommend that you use an account on the test server.

In [8]:
# Set the NDEx server and the user account.
# This "scratch" account will work for this tutorial, but you should replace it with your own account.
ndex_server = 'http://test.ndexbio.org'
ndex_user, ndex_pass = 'scratch', 'scratch'

In [47]:
# Upload ontology to NDEx. The string after "v2/network/" is a unique identifier, which is called the UUID, of the ontology in NDEx
url, _ = ont.to_ndex(ndex_server=ndex_server, ndex_user=ndex_user, ndex_pass=ndex_pass)
print(url)

http://dev2.ndexbio.org/v2/network/3bc543c6-68f5-11e8-9d1c-0660b7976219


In [48]:
# Download the ontology from NDEx
ont2 = Ontology.from_ndex(url)
print(ont2)


8 genes, 7 terms, 9 gene-term relations, 7 term-term relations
node_attributes: ['name', u'Vis:Shape', u'Vis:Border Paint', u'Vis:Size', u'Vis:Fill Color']
edge_attributes: [u'Vis:Visible']


# Inspecting the structure of an ontology

An Ontology object contains seven attributes:

* ``genes`` : List of gene names
* ``terms`` : List of term names
* ``gene_2_term`` : dictionary mapping a gene name to a list of terms connected to that gene. Terms are represented as their 0-based index in ``terms``.
* ``term_2_gene`` : dictionary mapping a term name to a list or genes connected to that term. Genes are represented as their 0-based index in ``genes``.
* ``child_2_parent`` : dictionary mapping a child term to its parent terms.
* ``parent_2_child`` : dictionary mapping a parent term to its children terms.
* ``term_sizes`` : A list of each term's size, i.e. the number of unique genes contained within this term and its descendants. The order of this list is the same as ``terms``. For every ``i``, it holds that ``term_sizes[i] = len(self.term_2_gene[self.terms[i]])``

In [49]:
ont.genes

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

In [50]:
ont.terms

['S0', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6']

In [51]:
ont.gene_2_term

{'A': [3],
 'B': [3],
 'C': [3, 4],
 'D': [4],
 'E': [5],
 'F': [5],
 'G': [6],
 'H': [6]}

In [52]:
ont.term_2_gene

{'S0': [],
 'S1': [],
 'S2': [],
 'S3': [0, 1, 2],
 'S4': [2, 3],
 'S5': [4, 5],
 'S6': [6, 7]}

In [53]:
ont.child_2_parent

{'S0': [],
 'S1': ['S0'],
 'S2': ['S0'],
 'S3': ['S1'],
 'S4': ['S1'],
 'S5': ['S2', 'S1'],
 'S6': ['S2']}

In [54]:
ont.parent_2_child

{'S0': ['S1', 'S2'],
 'S1': ['S3', 'S4', 'S5'],
 'S2': ['S5', 'S6'],
 'S3': [],
 'S4': [],
 'S5': [],
 'S6': []}

Alternatively, the hierarchical connections can be viewed as a binary matrix, using `Ontology.connected()`

In [55]:
conn = ont.connected()
np.array(conn, dtype=np.int32)

array([[1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1]], dtype=int32)

A summary of an Ontology’s object, i.e. the number of genes, terms, and connections, can be printed `print(ont)`

In [56]:
print(ont)

8 genes, 7 terms, 9 gene-term relations, 7 term-term relations
node_attributes: []
edge_attributes: []


# Manipulating the structure of an ontology

DDOT provides several convenience functions for processing Ontologies into a desirable structure. Currently, there are no functions for adding genes and terms. If this is needed, then we recommend creating a new Ontology or manipulating the contents in a different library, such as NetworkX or igraph, and transforming the results into Ontology.

### Renaming nodes

In [57]:
# Renaming genes and terms.
ont2 = ont.rename(genes={'A' : 'A_alias'}, terms={'S0':'S0_alias'})
ont2.to_table()

,Parent,Child,EdgeType
0,S2,S5,Child-Parent
1,S2,S6,Child-Parent
2,S1,S3,Child-Parent
3,S1,S4,Child-Parent
4,S1,S5,Child-Parent
5,S0_alias,S1,Child-Parent
6,S0_alias,S2,Child-Parent
7,S3,C,Gene-Term
8,S4,C,Gene-Term
9,S3,B,Gene-Term


### Delete S1 and G while preserving transitive connections

<img src="https://raw.githubusercontent.com/michaelkyu/ddot/master/docs/toy_ontology_delete_transitive.png" height="250" align="left">

In [58]:
ont2 = ont.delete(to_delete=['S1', 'G'])
print(ont2)

7 genes, 6 terms, 8 gene-term relations, 6 term-term relations
node_attributes: []
edge_attributes: []


### Delete S1 and G (don't preserve transitive connections)

<img src="https://raw.githubusercontent.com/michaelkyu/ddot/master/docs/toy_ontology_delete_not_transitive.png" height="250" align="left">

In [59]:
ont2 = ont.delete(to_delete=['S1', 'G'], preserve_transitivity=False)
print(ont2)

7 genes, 6 terms, 8 gene-term relations, 3 term-term relations
node_attributes: []
edge_attributes: []


### Propagate gene-term connections

* Often times, it is convenient to explicitly include all transitive connections in the hierarchy. That is, if a hierarchy has edges A-->B and B-->C, then the hierarchy also has A-->C. This can be done by calling `Ontology.propagate(direction='forward')` function.

* On the other hand, all transitive connections can be removed with `Ontology.propagate(direction='reverse')`. This is useful as a parsimonious set of connections.


<img src="https://raw.githubusercontent.com/michaelkyu/ddot/master/docs/toy_ontology_propagate_gene_term.png" height="250" align="left">

In [66]:
# Include all transitive connections between genes and terms
ont2 = ont.propagate(direction='forward', gene_term=True, term_term=False)
print(ont2)

# Remove all transitive connections between genes and terms, retaining only a parsimonious set of connections
ont3 = ont2.propagate(direction='reverse', gene_term=True, term_term=False)
print(ont3)

8 genes, 7 terms, 27 gene-term relations, 7 term-term relations
node_attributes: []
edge_attributes: []
8 genes, 7 terms, 9 gene-term relations, 7 term-term relations
node_attributes: []
edge_attributes: []


### Propagate term-term connections

<img src="https://raw.githubusercontent.com/michaelkyu/ddot/master/docs/toy_ontology_propagate_term_term.png" height="250" align="left">

In [67]:
# Include all transitive connections between terms
ont2 = ont.propagate(direction='forward', gene_term=False, term_term=True)
print(ont2)

# Remove all transitive connections between terms, retaining only a parsimonious set of connections
ont3 = ont2.propagate(direction='reverse', gene_term=False, term_term=True)
print(ont3)

8 genes, 7 terms, 9 gene-term relations, 11 term-term relations
node_attributes: []
edge_attributes: []
8 genes, 7 terms, 9 gene-term relations, 7 term-term relations
node_attributes: []
edge_attributes: []


### Take the subbranch consisting of all term and genes under S1

<img src="https://raw.githubusercontent.com/michaelkyu/ddot/master/docs/toy_ontology_S1.png" height="250" align="left">

In [70]:
ont2 = ont.focus(branches=['S1'])
print(ont2)

Genes and Terms to keep: 10
6 genes, 4 terms, 7 gene-term relations, 3 term-term relations
node_attributes: []
edge_attributes: []


# Inferring a data-driven ontology

* Given a set of genes and a gene similarity network, we can hierarchically cluster the genes to infer cellular subsystems using the CliXO algorithm. The resulting hierarchy of subsystems defines a "data-driven gene ontology". For more information about the CLIXO algorithm, see Kramer et al. Bioinformatics, 30(12), pp.i34-i42. 2014.
* Conversely, we can also "flatten" the ontology structure to infer a gene-by-gene similarity network. In particular, the similarity between two genes is calculated as the size of the smallest common subsystem, known as "Resnik semantic similarity".
* The CLIXO algorithm has been designed to reconstruct the original hierarchy from the Resnik score.

In [91]:
# Flatten ontology to gene-by-gene network

sim, genes = ont.flatten()
print('Similarity matrix')
print(sim)
print('Row/column names of similarity matrix')
print(genes)

Similarity matrix
[[ 1.41503751  1.41503751  1.41503751  0.41503751  0.41503751  0.41503751 -0.         -0.        ]
 [ 1.41503751  1.41503751  1.41503751  0.41503751  0.41503751  0.41503751 -0.         -0.        ]
 [ 1.41503751  1.41503751  2.          2.          0.41503751  0.41503751 -0.         -0.        ]
 [ 0.41503751  0.41503751  2.          2.          0.41503751  0.41503751 -0.         -0.        ]
 [ 0.41503751  0.41503751  0.41503751  0.41503751  2.          2.          1.          1.        ]
 [ 0.41503751  0.41503751  0.41503751  0.41503751  2.          2.          1.          1.        ]
 [-0.         -0.         -0.         -0.          1.          1.          2.          2.        ]
 [-0.         -0.         -0.         -0.          1.          1.          2.          2.        ]]
Row/column names of similarity matrix
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H']


In [92]:
# Reconstruct the ontology using the CLIXO algorithm.
# In general, you may feed any kind of gene-gene similarities, e.g. measurements of protein-protein interactions, gene co-expression, or genetic interactions.

ont2 = Ontology.run_clixo(sim, 0.0, 1.0, square=True, square_names=genes)
print(ont2)

8 genes, 7 terms, 9 gene-term relations, 7 term-term relations
node_attributes: []
edge_attributes: ['CLIXO_score']


In [93]:
ont2.to_table(edge_attr=True)

,Parent,Child,EdgeType,CLIXO_score
0,S:14,S:12,Child-Parent,0.000000
1,S:14,S:13,Child-Parent,0.000000
2,S:12,S:8,Child-Parent,1.000000
3,S:12,S:9,Child-Parent,1.000000
4,S:13,S:9,Child-Parent,0.415038
5,S:13,S:10,Child-Parent,0.415038
6,S:13,S:11,Child-Parent,0.415038
7,S:11,A,Gene-Term,1.415040
8,S:11,C,Gene-Term,1.415040
9,S:10,C,Gene-Term,2.000000


# Ontology alignment

* The structures of two ontologies can be compared through a procedure known as ontology alignment. Ontology.align() implements the ontology alignment described in (Dutkowski et al. Nature biotechnology, 31(1), 2013), in which terms are matched if they contain similar sets of genes and if their parents and children terms are also similar.

* Ontology alignment is particularly useful for annotating a data-driven gene ontology by aligning it to a curated ontology such as the Gene Ontology (GO). For instance, if a data-driven term is identified to have a similar set of genes as the GO term for DNA repair, then the data-driven subsystem can be annotated as being involved in DNA repair. Moreover, data-driven terms with no matches in the ontology alignment may represent new molecular mechanisms.

<img src="https://raw.githubusercontent.com/michaelkyu/ddot/master/docs/toy_ontology_alignment.png" height="250" align="left">

In [95]:
## Make a second ontology (the ontology to the right in the above diagram)

# Connections from child terms to parent terms
hierarchy = [('T3', 'T1'),
             ('T4', 'T1'),
             ('T1', 'T0'),
             ('T5', 'T0')]

# Connections from genes to terms
mapping = [('A', 'T3'),
           ('B', 'T3'),
           ('C', 'T3'),
           ('D', 'T4'),
           ('E', 'T4'),
           ('F', 'T4'),
           ('G', 'T5'),
           ('H', 'T5')]

# Construct ontology
ont_B = Ontology(hierarchy, mapping)

In [96]:
ont.align(ont_B)

collapse command: /cellar/users/mikeyu/DeepTranslate/ddot/ddot/alignOntology/collapseRedundantNodes /tmp/tmpJnX48w
collapse command: /cellar/users/mikeyu/DeepTranslate/ddot/ddot/alignOntology/collapseRedundantNodes /tmp/tmpV5cmAw
Alignment command: /cellar/users/mikeyu/DeepTranslate/ddot/ddot/alignOntology/calculateFDRs /tmp/tmpYAcsHr /tmp/tmpyikaVI 0.05 criss_cross /tmp/tmpM_lMdO 100 40 gene


,Term,Similarity,FDR
Term,,,
S3,T3,0.985294,0.000
S1,T1,0.913608,0.000
S6,T5,0.910000,0.030
S0,T0,0.892982,0.025


# Construct ontotypes

* A major goal of genetics is to understand how genotype translates to phenotype. An ontology represents biological structure through which this genotype-phenotype translation happens. 

* Given a set of mutations comprising a genotype, DDOT allows you to propagate the impact of these mutations to the subsystems containing these genes in the ontology. In particular, the impact on a subsystem is estimated by the number of its genes that have been mutated. These subsystem activities, which we have called an “ontotype”, enables more accurate and interpretable predictions of phenotype from genotype (Yu et al. Cell Systems 2016, 2(2), pp.77-88. 2016).

In [97]:
# Genotypes can be represented as tuples of mutated genes
genotypes = [('A', 'B'),
             ('A', 'E'),
             ('A', 'H'),
             ('B', 'E'),
             ('B', 'H'),
             ('C', 'F'),
             ('D', 'E'),
             ('D', 'H'),
             ('E', 'H'),
             ('G', 'H')]

# Calculate the ontotypes, represented a genotype-by-term matrix. Each value represents the functional impact on a term in a genotype.
ontotypes = ont.get_ontotype(genotypes)
print(ontotypes)

   S0  S1  S2  S3  S4  S5  S6
0   0   0   0   2   0   0   0
1   0   0   0   1   0   1   0
2   0   0   0   1   0   0   1
3   0   0   0   1   0   1   0
4   0   0   0   1   0   0   1
5   0   0   0   1   1   1   0
6   0   0   0   0   1   1   0
7   0   0   0   0   1   0   1
8   0   0   0   0   0   1   1
9   0   0   0   0   0   0   2


In [102]:
# Genotypes can also be represented a genotype-by-gene matrix as an alternative input format
import pandas as pd, numpy as np
genotypes_df = pd.DataFrame(np.zeros((len(genotypes), len(ont.genes)), np.float64),
                            index=['Genotype%s' % i for i in range(len(genotypes))],
                            columns=ont.genes)
for i, (g1, g2) in enumerate(genotypes):
    genotypes_df.loc['Genotype%s' % i, g1] = 1.0
    genotypes_df.loc['Genotype%s' % i, g2] = 1.0
print('Genotype matrix')
print(genotypes_df)
print("")

ontotypes = ont.get_ontotype(genotypes_df, input_format='matrix')
print('Ontotype matrix')
print(ontotypes)

Genotype matrix
             A    B    C    D    E    F    G    H
Genotype0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
Genotype1  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
Genotype2  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
Genotype3  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0
Genotype4  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0
Genotype5  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0
Genotype6  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0
Genotype7  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0
Genotype8  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0
Genotype9  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0

Ontotype matrix
            S0   S1   S2   S3   S4   S5   S6
Genotype0  0.0  0.0  0.0  2.0  0.0  0.0  0.0
Genotype1  0.0  0.0  0.0  1.0  0.0  1.0  0.0
Genotype2  0.0  0.0  0.0  1.0  0.0  0.0  1.0
Genotype3  0.0  0.0  0.0  1.0  0.0  1.0  0.0
Genotype4  0.0  0.0  0.0  1.0  0.0  0.0  1.0
Genotype5  0.0  0.0  0.0  1.0  1.0  1.0  0.0
Genotype6  0.0  0.0  0.0  0.0  1.0  1.0  0.0
Genotype7  0.0  0.0  0.0  0.0  1.0  0.0  1.0
Genotype8  0

# Conversions to NetworkX and igraph

In [103]:
# Convert to an igraph object
G = ont.to_igraph()
print(G)

IGRAPH DN-- 15 16 --
+ attr: NodeType (v), name (v), EdgeType (e)
+ edges (vertex names):
A->S3, B->S3, C->S3, C->S4, D->S4, E->S5, F->S5, G->S6, H->S6, S5->S2, S6->S2,
S3->S1, S4->S1, S5->S1, S1->S0, S2->S0


In [104]:
# Reconstruct the Ontology object from the igraph object
Ontology.from_igraph(G)

8 genes, 7 terms, 9 gene-term relations, 7 term-term relations
node_attributes: []
edge_attributes: []

In [105]:
# Convert to a NetworkX object
G = ont.to_networkx()
print(G.nodes())
print(G.edges())

['A', 'C', 'B', 'E', 'D', 'G', 'F', 'S3', 'H', 'S1', 'S0', 'S6', 'S5', 'S4', 'S2']
[('A', 'S3'), ('C', 'S3'), ('C', 'S4'), ('B', 'S3'), ('E', 'S5'), ('D', 'S4'), ('G', 'S6'), ('F', 'S5'), ('S3', 'S1'), ('H', 'S6'), ('S1', 'S0'), ('S6', 'S2'), ('S5', 'S2'), ('S5', 'S1'), ('S4', 'S1'), ('S2', 'S0')]


In [106]:
# Reconstruct the Ontology object from the NetworkX object
tmp = Ontology.from_networkx(G)
print(tmp)

8 genes, 7 terms, 9 gene-term relations, 7 term-term relations
node_attributes: []
edge_attributes: []


# Ontology visualization using HiView (http://hiview.ucsd.edu)
* HiView is a web application for general visualization of the hierarchical structure in ontologies.
* To use HiView, you must first upload your ontology into NDEx using the [Ontology.to_ndex()](http://ddot.readthedocs.io/en/latest/ontology.html#ddot.Ontology.to_ndex) function, and then input the NDEx URL for the ontology to HiView
* In contrast to almost all other hierarchical visualization tools, which are limited to simple tree structures, HiView also supports more complicated hierarchies in the form of directed acyclic graphs, in which nodes may have multiple parents.

### A simple upload to NDEx and visualization in HiView
* Upload ontologies to NDEx using the `Ontology.to_ndex()` function.
* Setting the parameter `layout="bubble"` (default value) will identify a spanning tree of the DAG and then lay this tree in a space-compact manner. When viewing in HiView, only the edges in the spanning tree are shown, while the other edges can be chosen to be shown.

<img src="https://raw.githubusercontent.com/michaelkyu/ddot/master/docs/toy_ontology_spanning_tree.png" height="250" align="left">

In [107]:
url, _ = ont.to_ndex(ndex_server=ndex_server,
                     ndex_user=ndex_user,
                     ndex_pass=ndex_pass,
                     layout='bubble')

print('Go to http://hiview.ucsd.edu in your web browser')
print('Enter this into the "NDEx Sever URL" field: %s' % ndex_server)
print('Enter this into the "UUID of the main hierarchy" field: %s' % url.split('/')[-1])

Go to http://hiview.ucsd.edu in your web browser
Enter this into the "NDEx Sever URL" field: http://test.ndexbio.org
Enter this into the "UUID of the main hierarchy" field: 28e8a289-6943-11e8-9d1c-0660b7976219


### An alternative layout by duplicating nodes
* Setting the parameter `layout="bubble-collect"` will convert the DAG into a tree by duplicating nodes.
* This transformation enables the ontology structure to be visualized without edges crossing.

<img src="https://raw.githubusercontent.com/michaelkyu/ddot/master/docs/toy_ontology_dag2tree.png" height="250" align="left">

In [97]:
url, _ = ont.to_ndex(ndex_server=ndex_server,
                     ndex_user=ndex_user,
                     ndex_pass=ndex_pass,
                     layout='bubble-collect')

print('Go to http://hiview.ucsd.edu in your web browser')
print('Enter this into the "NDEx Sever URL" field: %s' % ndex_server)
print('Enter this into the "UUID of the main hierarchy" field: %s' % url.split('/')[-1])

Go to http://hiview.ucsd.edu in your web browser
Enter this into the "NDEx Sever URL" field: http://test.ndexbio.org
Enter this into the "UUID of the main hierarchy" field: e6b6be56-5fe7-11e8-9d1c-0660b7976219


### Visualizing metadata by modifying node labels, colors, and sizes

* An Ontology object has a `node_attr` field that is a pandas DataFrame. The rows of the dataframe are genes or terms, and the columns are node attributes.
* HiView understands special node attributes to control the node labels, colors, and sizes.

In [121]:
# Set the node labels (default is the gene and term names, as found in Ontology.genes and Ontology.terms)
ont.node_attr.loc['S4', 'Label'] = 'S4 alias'
ont.node_attr.loc['S5', 'Label'] = 'S5 alias'

# Set the fill color of nodes
ont.node_attr.loc['C', 'Vis:Fill Color'] = '#7fc97f'
ont.node_attr.loc['S1', 'Vis:Fill Color'] = '#beaed4'
ont.node_attr.loc['S0', 'Vis:Fill Color'] = '#fdc086'

# Set the node sizes (if not set, the default is the term size, as found in Ontology.term_sizes)
ont.node_attr.loc['C', 'Size'] = 10

ont.node_attr

,Label,Vis:Fill Color,Size
Node,,,
S4,S4 alias,NaN,NaN
S5,S5 alias,NaN,NaN
C,NaN,#7fc97f,10.0
S1,NaN,#beaed4,NaN
S0,NaN,#fdc086,NaN


In [119]:
url, _ = ont.to_ndex(ndex_server=ndex_server,
                     ndex_user=ndex_user,
                     ndex_pass=ndex_pass,
                     layout='bubble-collect')

print('Go to http://hiview.ucsd.edu in your web browser')
print('Enter this into the "NDEx Sever URL" field: %s' % ndex_server)
print('Enter this into the "UUID of the main hierarchy" field: %s' % url.split('/')[-1])

Go to http://hiview.ucsd.edu in your web browser
Enter this into the "NDEx Sever URL" field: http://test.ndexbio.org
Enter this into the "UUID of the main hierarchy" field: 6f790174-6988-11e8-9d1c-0660b7976219


In [125]:
# Clear node attributes (optional)
ont.clear_node_attr()
ont.node_attr

""
Node


### Visualize gene-gene interaction networks alongside the ontology
* Every term in an ontology represents a biological function shared among the term's genes. Based on this intuition, those genes should be interacting in different ways, e.g. protein-protein interactions, RNA expression, or genetic interactions.
* Gene-gene interaction networks can be uploaded with the ontology to NDEx, so that they can be visualized at the same time in HiView

In [4]:
# Calculate a gene-by-gene similarity matrix using the Resnik semantic similarity definition (see section "Inferring a data-driven ontology")
sim, genes = ont.flatten()
print(genes)
print(sim)

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H']
[[ 1.4150375  1.4150375  1.4150375  0.4150375  0.4150375  0.4150375 -0.        -0.       ]
 [ 1.4150375  1.4150375  1.4150375  0.4150375  0.4150375  0.4150375 -0.        -0.       ]
 [ 1.4150375  1.4150375  2.         2.         0.4150375  0.4150375 -0.        -0.       ]
 [ 0.4150375  0.4150375  2.         2.         0.4150375  0.4150375 -0.        -0.       ]
 [ 0.4150375  0.4150375  0.4150375  0.4150375  2.         2.         1.         1.       ]
 [ 0.4150375  0.4150375  0.4150375  0.4150375  2.         2.         1.         1.       ]
 [-0.        -0.        -0.        -0.         1.         1.         2.         2.       ]
 [-0.        -0.        -0.        -0.         1.         1.         2.         2.       ]]


In [5]:
# Convert the gene-by-gene similarity matrix into a dataframe with a "long" format, where rows represent gene pairs. This conversion can be easily done with ddot.melt_square()
import pandas as pd
sim_df = pd.DataFrame(sim, index=genes, columns=genes)
sim_long = ddot.melt_square(sim_df)
sim_long.head()

,Gene1,Gene2,similarity
0,A,B,1.415038
1,A,C,1.415038
2,A,D,0.415038
3,A,E,0.415038
4,A,F,0.415038


In [6]:
# Create other gene-gene interactions. For example, these can represent protein-protein interactions or gene co-expression. Here, we simulate interactions by adding a random noise to the Resnik similarity
sim_long['example_interaction_type1'] = sim_long['similarity'] + np.random.random(sim_long.shape[0]) / 2.
sim_long['example_interaction_type2'] = sim_long['similarity'] + np.random.random(sim_long.shape[0]) / 2.
sim_long.head()

,Gene1,Gene2,similarity,example_interaction_type1,example_interaction_type2
0,A,B,1.415038,1.532573,1.610435
1,A,C,1.415038,1.570824,1.627529
2,A,D,0.415038,0.563761,0.482697
3,A,E,0.415038,0.730091,0.857810
4,A,F,0.415038,0.606655,0.517407


In [9]:
# Include the above gene-gene interactions by setting the `network` and `main_feature` parameters.
url, _ = ont.to_ndex(ndex_server=ndex_server,
                     ndex_user=ndex_user,
                     ndex_pass=ndex_pass,
                     network=sim_long,
                     main_feature='similarity',
                     layout='bubble-collect',
                     verbose=True)

print('Go to http://hiview.ucsd.edu in your web browser')
print('Enter this into the "NDEx Sever URL" field: %s' % ndex_server)
print('Enter this into the "UUID of the main hierarchy" field: %s' % url.split('/')[-1])

Uploading 7 terms
0 Term: S0 Gene pairs: 28 Genes: 8 Time: 0.1695 Upload time: 0.1615 NDEx URL: http://dev2.ndexbio.org/v2/network/3abffde9-6993-11e8-9d1c-0660b7976219
1 Term: S1 Gene pairs: 15 Genes: 6 Time: 0.1679 Upload time: 0.1591 NDEx URL: http://dev2.ndexbio.org/v2/network/3ad9c77c-6993-11e8-9d1c-0660b7976219
2 Term: S2 Gene pairs: 6 Genes: 4 Time: 0.1651 Upload time: 0.1577 NDEx URL: http://dev2.ndexbio.org/v2/network/3af342ee-6993-11e8-9d1c-0660b7976219
3 Term: S3 Gene pairs: 3 Genes: 3 Time: 0.1638 Upload time: 0.1599 NDEx URL: http://dev2.ndexbio.org/v2/network/3b0c2220-6993-11e8-9d1c-0660b7976219
4 Term: S4 Gene pairs: 1 Genes: 2 Time: 0.1579 Upload time: 0.1541 NDEx URL: http://dev2.ndexbio.org/v2/network/3b24da42-6993-11e8-9d1c-0660b7976219
5 Term: S5 Gene pairs: 1 Genes: 2 Time: 0.1609 Upload time: 0.1571 NDEx URL: http://dev2.ndexbio.org/v2/network/3b3d4448-6993-11e8-9d1c-0660b7976219
6 Term: S6 Gene pairs: 1 Genes: 2 Time: 0.1649 Upload time: 0.1608 NDEx URL: http://de